In [18]:
import pickle
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from dotenv import load_dotenv
import os
from spotipy.exceptions import SpotifyException

## Load track_infos from pickle file

In [2]:
with open("unique_track.pickle", "rb") as f:
    track_infos = pickle.load(f)

len(track_infos)

2262292

In [3]:
track_infos

{'0UaMYEvWZi0ZqiDOoHU3YI': {'artist_name': 'Missy Elliott',
  'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)'},
 '6I9VzXrHxO9rA9A5euc8Ak': {'artist_name': 'Britney Spears',
  'track_name': 'Toxic'},
 '0WqIKmW4BTrj3eJFmnCKMv': {'artist_name': 'Beyoncé',
  'track_name': 'Crazy In Love'},
 '1AWQoqb9bSvzTjaLralEkT': {'artist_name': 'Justin Timberlake',
  'track_name': 'Rock Your Body'},
 '1lzr43nnXAijIGYnCT8M8H': {'artist_name': 'Shaggy',
  'track_name': "It Wasn't Me"},
 '0XUfyU2QviPAs6bxSpXYG4': {'artist_name': 'Usher', 'track_name': 'Yeah!'},
 '68vgtRHr7iZHpzGpon6Jlo': {'artist_name': 'Usher', 'track_name': 'My Boo'},
 '3BxWKCI06eQ5Od8TY2JBeA': {'artist_name': 'The Pussycat Dolls',
  'track_name': 'Buttons'},
 '7H6ev70Weq6DdpZyyTmUXk': {'artist_name': "Destiny's Child",
  'track_name': 'Say My Name'},
 '2PpruBYCo4H7WOBJ7Q2EwM': {'artist_name': 'OutKast',
  'track_name': 'Hey Ya! - Radio Mix / Club Mix'},
 '2gam98EZKrF9XuOkU13ApN': {'artist_name': 'Nelly Furtado',
  'track_nam

- Create client 

In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [6]:
cli_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=cli_credentials_manager)
sp.audio_features("644qZVMUMsj2UkVPITKtwV")


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=644qZVMUMsj2UkVPITKtwV:
 Max Retries, reason: too many 429 error responses

In [7]:
invalid_tracks = []

- Get audio features using spotify API (Example: sp.audio_features("6I9VzXrHxO9rA9A5euc8Ak"))

In [19]:
def get_and_update_feats(batch_ids: list):
    unneccesary_feats = ["id", "uri", "track_href", "analysis_url", "type"] # Unneccesary features
    feature_list = sp.audio_features(batch_ids)
    
    for i, feats in enumerate(feature_list, start=0):
        if (feats is None):
            invalid_tracks.append(batch_ids[i])
            print(f"Track with ID {batch_ids[i]}'s features is None")
            continue
        
        id = batch_ids[i]
        for feat in unneccesary_feats: # Remove unnecessary features
            feats.pop(feat, None)
    
        temp = {
            "audio_feats":feats
        }

        track_infos[id].update(temp)


- Get and update all audio features into track_infos

In [9]:
keys = list(track_infos.keys())

- update original batch

In [10]:
current_track = None
def update_with_batch(start_index, end_index, keys, batch_size: int = 100):
    # max_size = end_index
    i = start_index
    while (True):
        print(f"Current tracks updated: {i}")
        
        if (i >= end_index):
            i = end_index - batch_size - 1
            # print("Here: ", i)


        batch = keys[i:i+batch_size]
        get_and_update_feats(batch)
        
        time.sleep(2)

        if (i == end_index - 1):
            break
        
        i += batch_size
    
    

In [11]:
k_temps = list(keys)
print(k_temps[1697067])
track_infos[k_temps[207651]]

644qZVMUMsj2UkVPITKtwV


{'artist_name': 'Ariana Grande', 'track_name': 'Put Your Hearts Up'}

- Dump to pickle again

In [17]:
start_index = int(input("Enter start index to split: "))
end_index = int(input("Enter end index to split: "))
update_with_batch(start_index, end_index, keys)


Current tracks updated: 200


Max Retries reached


Wait tmr
Edited tracks: 200


In [ ]:
def split_pickle(track_infos, start_index: int, end_index:int):
    keys = list(track_infos.keys())
    temp = dict()

    for i in range(start_index, end_index):
        if (keys[i] in invalid_tracks):
            continue
        temp[keys[i]] = track_infos[keys[i]]

    return temp

In [ ]:
split_start_indx = int(input())
split_end_indx = int(input())
pkl_name = f"track_infos_{split_start_indx}_{split_end_indx}.pickle"
temp_pickle = split_pickle(track_infos, split_start_indx, split_end_indx)

with open(pkl_name, "wb") as f:
    pickle.dump(temp_pickle, f)

ValueError: invalid literal for int() with base 10: ''

---

In [ ]:
try:
    a = sp.audio_features("6I9VzXrHxO9rA9A5euc8Ak")[0]
except:
    print("Max entries")

Max Retries reached


Max entries
